# Sagemaker 

## Transformar los datos y entrenar un modelo dentro de un notebook Jupyter.

El kernel utilizado en este notebook es `Python 3 (TensorFlow 2.6 Python 3.8 CPU Optimized`




En este notebook demostraremos un viaje hacia el aprendizaje automático nativo en la nube, empezando por un enfoque más tradicional de desarrollo de modelos y entrenamiento directamente en notebooks Jupyter, pasando por transformaciones de datos gestionadas de forma remota y entrenamiento con Amazon SageMaker, hasta pipelines totalmente automatizados con [SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines/).

En este primer cuaderno predeciremos el precio de la vivienda basándonos en el conocido [Boston Housing dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) con un modelo de regresión simple en Tensorflow 2. Este Dataset público contiene 13 features relativas al parque de viviendas de las ciudades del área de Boston. Las features incluyen el número medio de habitaciones, la accesibilidad a las carreteras radiales, la adyacencia a un río importante, etc.  

Para empezar, crearemos directorios para los datos de entrenamiento y de prueba.  También configuraremos una sesión de SageMaker para realizar varias operaciones, y especificaremos un cubo de Amazon S3 para mantener los datos de entrada y salida.  El cubo por defecto utilizado aquí es creado por SageMaker si no existe ya, y nombrado de acuerdo con el ID de la cuenta de AWS y la región de AWS.  

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
import os

data_dir = os.path.join(os.getcwd(), 'data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'data/test')
os.makedirs(test_dir, exist_ok=True)

raw_dir = os.path.join(os.getcwd(), 'data/raw')
os.makedirs(raw_dir, exist_ok=True)

batch_dir = os.path.join(os.getcwd(), 'data/batch')
os.makedirs(batch_dir, exist_ok=True)

# Exploratory Data Analysis (EDA)

According to The [State of Data Science 2020](https://www.anaconda.com/state-of-data-science-2020) survey, data management, exploratory data analysis (EDA), feature selection, and feature engineering accounts for more than 66% of a data scientist’s time.

Exploratory Data Analysis is an approach in analyzing data sets to summarize their main characteristics, often using statistical graphics and other data visualization methods.
EDA assists Data science professionals in various ways:-

- Getting a better understanding of data.
- Identifying various data patterns.
- Getting a better understanding of the problem statement.

Numerical EDA gives you some very important information, such as the names and data types of the columns, and the dimensions of the DataFrame. 
Visual EDA on the other hand will give you insight into features and target relationship and distribution.

First we'll load the Boston Housing dataset and explore the data.

Según la encuesta [State of Data Science 2020](https://www.anaconda.com/state-of-data-science-2020), la gestión de datos, el análisis exploratorio de datos (EDA), la selección de features y la ingeniería de features representan más del 66% del tiempo de un científico de datos.

El análisis exploratorio de datos es un enfoque para analizar Datasets con el fin de resumir sus principales características, a menudo utilizando gráficos estadísticos y otros métodos de visualización de datos.
El EDA ayuda a los profesionales de la ciencia de los datos de varias maneras

- Obtener una mejor comprensión de los datos.
- Identificar patrones en los datos.
- Obtener una mejor comprensión del planteamiento del problema.

El EDA numérico da información muy importante, como los nombres y tipos de datos de las columnas, y las dimensiones del DataFrame.
El EDA visual, por otro lado, da una visión de la relación entre las features y el target y de la distribución de las muestras.


Primero cargaremos el Dataset de viviendas de Boston y exploraremos los datos.

In [ ]:
from tensorflow.python.keras.datasets import boston_housing

# El Dataset ya tiene un split hecho al cargarse
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
columns=['CRIM',
         'ZN',
         'INDUS',
         'CHAS',
         'NOX',
         'RM',
         'AGE',
         'DIS',
         'RAD',
         'TAX',
         'PTRATIO',
         'B',
         'LSTAT']

df = pd.DataFrame(x_train, columns=columns)
df["MEDV"] = y_train
df

### EDA numérico

Comprobamos qué tamaño tiene el Dataset, cuántas features tiene y de qué tipo, y cuál es el target.

In [ ]:
df.info()

Hay 14 atributos en cada caso del Dataset:

1. CRIM - tasa de criminalidad per cápita por ciudad.
2. ZN - proporción de terrenos residenciales agrupados en solares de más de 25.000 pies cuadrados.
3. INDUS - proporción de acres comerciales no minoristas por ciudad.
4. CHAS - Variable ficticia del río Charles (1 si el tramo limita con el río; 0 en caso contrario).
5. NOX - concentración de óxidos nítricos (partes por 10 millones).
6. RM - número medio de habitaciones por vivienda.
7. EDAD - proporción de unidades habitadas por sus propietarios construidas antes de 1940.
8. DIS - distancias ponderadas a cinco centros de empleo de Boston.
9. RAD - índice de accesibilidad a las autopistas radiales.
10. TAX - tasa de impuesto sobre el valor total de la propiedad por cada 10.000 dólares.
11. PTRATIO - ratio alumno-profesor por ciudad.
12. B - 1000(Bk - 0.63)^2 donde Bk es la proporción de gente de color por ciudad.
13. LSTAT - % de estatus inferior de la población.
14. MEDV - Valor medio de las viviendas habitadas por sus propietarios.

Ahora, vamos a resumir los datos para ver la distribución de los mismos

In [ ]:
df.describe()

### EDA visual

Intentemos encontrar el nivel de delincuencia en relación con la condición de ciudadano y el valor medio de las viviendas.

In [ ]:
df['medv_bins'] = pd.cut(df.MEDV, bins=5, include_lowest=True)
df.medv_bins.head()

In [ ]:
df['lstat_bins'] = pd.cut(df.LSTAT, bins=[0, 7, 17, 38], labels=['richest', 'ordinary', 'poorest'], include_lowest=True)
df.lstat_bins.head()

In [ ]:
f, (ax_viol, ax_box) = plt.subplots(2, sharex=True)

sns.boxplot(x='medv_bins', y='CRIM', data=df, ax=ax_viol)
sns.swarmplot(x='medv_bins', y='CRIM', hue='lstat_bins', data=df, ax=ax_box)

ax_viol.set(xlabel='')

Podemos ver que a medida que el estatus de los ciudadanos es más pobre (LSTAT) y el valor medio de las casas es más bajo (MEDV), el nivel de delincuencia está creciendo (CRIM).

# Dataset transformation <a class="anchor" id="SageMakerProcessing">


A continuación, transformaremos el Dataset. En un flujo de trabajo típico de SageMaker, los notebooks sólo se utilizan para la creación de prototipos y pueden ejecutarse en instancias relativamente baratas y menos potentes, mientras que las tareas de procesamiento, formación y alojamiento de modelos se ejecutan en instancias independientes y más potentes gestionadas por SageMaker.



Ahora guardaremos los datos de features en bruto, y también guardaremos las etiquetas para el entrenamiento y las pruebas.

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

np.save(os.path.join(raw_dir, 'x_train.npy'), x_train)
np.save(os.path.join(raw_dir, 'x_test.npy'), x_test)
np.save(os.path.join(raw_dir, 'y_train.npy'), y_train)
np.save(os.path.join(raw_dir, 'y_test.npy'), y_test)

A continuación, ejecutaremos el preprocesamiento de datos como se muestra a continuación.

In [ ]:
import glob
import numpy as np
import os
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
x_train = np.load(os.path.join(raw_dir, 'x_train.npy'))
scaler.fit(x_train)

In [ ]:
input_files = glob.glob('{}/raw/*.npy'.format(data_dir))
print('\nINPUT FILE LIST: \n{}\n'.format(input_files))
for file in input_files:
    raw = np.load(file)
    # solo transformamos las columnas de features
    if 'y_' not in file:
        transformed = scaler.transform(raw)
    if 'train' in file:
        if 'y_' in file:
            output_path = os.path.join(train_dir, 'y_train.npy')
            np.save(output_path, raw)
            print('SAVED LABEL TRAINING DATA FILE\n')
        else:
            output_path = os.path.join(train_dir, 'x_train.npy')
            np.save(output_path, transformed)
            print('SAVED TRANSFORMED TRAINING DATA FILE\n')
    else:
        if 'y_' in file:
            output_path = os.path.join(test_dir, 'y_test.npy')
            np.save(output_path, raw)
            print('SAVED LABEL TEST DATA FILE\n')
        else:
            output_path = os.path.join(test_dir, 'x_test.npy')
            np.save(output_path, transformed)
            print('SAVED TRANSFORMED TEST DATA FILE\n')

#  Training <a class="anchor" id="SageMakerHostedTraining">


Ahora que hemos preparado un conjunto de datos, podemos pasar al entrenamiento del modelo.

In [ ]:
import numpy as np
import os
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def get_train_data(train_dir):
    x_train = np.load(os.path.join(train_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(train_dir, 'y_train.npy'))
    print('x train', x_train.shape,'y train', y_train.shape)

    return x_train, y_train


def get_test_data(test_dir):
    x_test = np.load(os.path.join(test_dir, 'x_test.npy'))
    y_test = np.load(os.path.join(test_dir, 'y_test.npy'))
    print('x test', x_test.shape,'y test', y_test.shape)

    return x_test, y_test

def get_model():
    inputs = tf.keras.Input(shape=(13,))
    hidden_1 = tf.keras.layers.Dense(13, activation='tanh')(inputs)
    hidden_2 = tf.keras.layers.Dense(6, activation='sigmoid')(hidden_1)
    outputs = tf.keras.layers.Dense(1)(hidden_2)
    return tf.keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
x_train, y_train = get_train_data(train_dir)
x_test, y_test = get_test_data(test_dir)

device = '/cpu:0'
print(device)
batch_size = 128
epochs = 80
learning_rate = 0.01
print('batch_size = {}, epochs = {}, learning rate = {}'.format(batch_size, epochs, learning_rate))

with tf.device(device):
    model = get_model()
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
              validation_data=(x_test, y_test))

    # evaluar en el Dataset
    scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
    print("\nTest MSE :", scores)


El archivo descomprimido debe incluir los activos requeridos por TensorFlow Serving para cargar el modelo y servirlo, incluyendo un archivo .pb:  

In [ ]:
model.save('model' + '/1')

# Puntuación del modelo

In [ ]:
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model('model/1')

x_test = np.load(os.path.join(test_dir, 'x_test.npy'))
y_test = np.load(os.path.join(test_dir, 'y_test.npy'))
scores = model.evaluate(x_test, y_test, verbose=2)
print("\nTest MSE :", scores)